In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 52.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 96.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 25.4 MB/s eta 0:00:00


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
cd /content/drive/MyDrive/text_classification/dot/word2vec-gpt

/content/drive/MyDrive/text_classification/dot/word2vec-gpt


In [19]:
import pandas as pd
from transformers import GPT2Tokenizer, GPT2Model
import torch
import numpy as np
import jieba

In [20]:
## gpt ##
# Read the Excel file
data = pd.read_excel('./data.xlsx')
sentences = data['review'].tolist()

def txt_cut(s):
    res = [w for w in jieba.lcut(s) if w.strip()]
    return " ".join(res)

# 对文本进行分词
tokenized_text = []
for i in sentences:
    new = txt_cut(i).split(' ')
    tokenized_text.extend(new)

# Initialize GPT-2 tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2Model.from_pretrained('gpt2')

# Create an empty list to store word vectors
word_vectors = []

# Iterate through each row in the list
for word in tokenized_text:

    # Encode the word with the tokenizer
    inputs = tokenizer(word, return_tensors='pt')

    # Get the model's output for the word
    outputs = model(**inputs)
    last_hidden_state = outputs.last_hidden_state

    # Append the word vector to the list
    word_vector = last_hidden_state[0, 0, :]

    word_vectors.append((word, word_vector))

Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
DEBUG:jieba:Dumping model to file cache /tmp/jieba.cache
Loading model cost 4.227 seconds.
DEBUG:jieba:Loading model cost 4.227 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


In [ ]:
# for word_vector in enumerate(word_vectors):
#     print(f"Word: {word_vector[1][0]}, Word Vector: {word_vector[1][1]}")

In [22]:
import pandas as pd
from gensim.models import Word2Vec
import jieba

In [23]:
## word2vec ##

# 初始化Word2Vec模型
model_w = Word2Vec(vector_size=100, window=5, min_count=1)

# 构建词汇表
model_w.build_vocab([tokenized_text])

# 训练模型
model_w.train([tokenized_text], total_examples=model_w.corpus_count, epochs=model_w.epochs)

# 创建一个空的DataFrame来保存词汇和对应的词向量
word_vector_df = pd.DataFrame()

for wd in model_w.wv.index_to_key:
    # 将词向量转化为Series，然后添加到DataFrame中
    word_vector_series = pd.Series([wd] + list(model_w.wv[wd]))
    word_vector_df = word_vector_df.append(word_vector_series, ignore_index=True)

<ipython-input-23-f084292095fa>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  word_vector_df = word_vector_df.append(word_vector_series, ignore_index=True)
<ipython-input-23-f084292095fa>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  word_vector_df = word_vector_df.append(word_vector_series, ignore_index=True)
<ipython-input-23-f084292095fa>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  word_vector_df = word_vector_df.append(word_vector_series, ignore_index=True)
<ipython-input-23-f084292095fa>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  word_vector_df = word_vector_df.append(word_vector_series, ignore_index=True)
<ipy

In [40]:
new_word_vectors = []

for word_vector in enumerate(word_vectors):
    word_vector_g = word_vector[1][1].detach().numpy()
    word_vector_g = word_vector_g.reshape(-1, 1)

    ## word2vec
    word_vector_w = model_w.wv[word_vector[1][0]]
    word_vector_w = word_vector_w.reshape(-1, 1)

    ## 新词向量
    new_word_vector = np.multiply(word_vector_g, word_vector_w.T)
    new_word_vectors.append((word, new_word_vector))

In [43]:
# 指定保存文件路径
save_path = 'new_word_vectors.txt'

# 保存单词和新词向量到文件
with open(save_path, 'w') as f:
    for word, word_vector in new_word_vectors:
        word_vector_str = ','.join(map(str, word_vector))
        line = f"{word}: {word_vector_str}\n"
        f.write(line)